In [8]:
import requests
import pandas as pd
import numpy as np
import folium
from folium.plugins import MiniMap
import requests
import folium
import collections
 

##카카오 API
def whole_region(keyword, start_x,start_y,end_x,end_y):
    #print(start_x,start_y,end_x,end_y)
    page_num = 1
    # 데이터가 담길 리스트
    all_data_list = []
    
    while(1):
        url = 'https://dapi.kakao.com/v2/local/search/keyword.json'
        params = {'query': keyword,'page': page_num, 
                 'rect': f'{start_x},{start_y},{end_x},{end_y}'}
        headers = {"Authorization": "KakaoAK 96acd1c64b18ad9e58aa41fde0add075"}
        ## 입력예시 -->> headers = {"Authorization": "KakaoAK f64acbasdfasdfasf70e4f52f737760657"}
        resp = requests.get(url, params=params, headers=headers)

        search_count = resp.json()['meta']['total_count']
        print('총 개수',search_count)
        
        if search_count > 45:
            print('좌표 4등분')
            dividing_x = (start_x + end_x) / 2
            dividing_y = (start_y + end_y) / 2
            ## 4등분 중 왼쪽 아래
            all_data_list.extend(whole_region(keyword, start_x,start_y,dividing_x,dividing_y))
            ## 4등분 중 오른쪽 아래
            all_data_list.extend(whole_region(keyword, dividing_x,start_y,end_x,dividing_y))
            ## 4등분 중 왼쪽 위
            all_data_list.extend(whole_region(keyword, start_x,dividing_y,dividing_x,end_y))
            ## 4등분 중 오른쪽 위
            all_data_list.extend(whole_region(keyword, dividing_x,dividing_y,end_x,end_y))
            return all_data_list
        
        else:
            if resp.json()['meta']['is_end']:
                all_data_list.extend(resp.json()['documents'])
                return all_data_list
            # 아니면 다음 페이지로 넘어가서 데이터 저장
            else:
                print('다음페이지')
                page_num += 1
                all_data_list.extend(resp.json()['documents'])
 

def overlapped_data(keyword, start_x, start_y, next_x, next_y, num_x, num_y):
    # 최종 데이터가 담길 리스트
    overlapped_result = []

    # 지도를 사각형으로 나누면서 데이터 받아옴
    for i in range(1,num_x+1):   ## 1,10
        end_x = start_x + next_x
        initial_start_y = start_y
        for j in range(1,num_y+1):  ## 1,6
            end_y = initial_start_y + next_y
            each_result= whole_region(keyword, start_x,initial_start_y,end_x,end_y)
            overlapped_result.extend(each_result)
            initial_start_y = end_y
        start_x = end_x
    
    return overlapped_result
 

def make_map(dfs):
    # 지도 생성하기
    m = folium.Map(location=[33.4935,126.6266],   # 기준좌표: 제주어딘가
                   zoom_start=12)

    # 미니맵 추가
    minimap = MiniMap() 
    m.add_child(minimap)

    # 마커
    for i in range(len(dfs)):
        folium.Marker([df['Y'][i],df['X'][i]],    # 제주 어딘가
                  tooltip=dfs['stores'][i],
                  popup=dfs['place_url'][i],
                  ).add_to(m)
    return m

In [9]:
# 시작 x 좌표 및 증가값
keyword = '게스트하우스'
start_x = 126.12484043304154
start_y = 33.18739686999
# 애월읍 좌표 33.45845507281501, 126.3634555492629
next_x = 0.01
next_y = 0.01
num_x = 200
num_y = 100
 

overlapped_result = overlapped_data(keyword, start_x, start_y, next_x, next_y, num_x, num_y)

# 최종 데이터가 담긴 리스트 중복값 제거
results = list(map(dict, collections.OrderedDict.fromkeys(tuple(sorted(d.items())) for d in overlapped_result)))
X = []
Y = []
stores = []
road_address = []
place_url = []
ID = []
for place in results:
    
    X.append(float(place['x']))
    Y.append(float(place['y']))
    stores.append(place['place_name'])
    road_address.append(place['road_address_name'])
    place_url.append(place['place_url'])
    ID.append(place['id'])

    ar = np.array([ID,stores, X, Y, road_address,place_url]).T
    df = pd.DataFrame(ar, columns = ['ID','stores', 'X', 'Y','road_address','place_url'])
    
print('total_reuslt_number = ',len(df))
 

display(df.head())
 



총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0

총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 1
총 개수 7
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 1
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 4
총 개수 3
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 2
총 개수 8
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0

총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 6
총 개수 1
총 개수 0
총 개수 1
총 개수 1
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 1
총 개수 3
총 개수 3
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0

총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 1
총 개수 2
총 개수 2
총 개수 0
총 개수 1
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 2
총 개수 4
총 개수 0
총 개수 0
총 개수 1
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0

총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 5
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 1
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 1
총 개수 0
총 개수 0
총 개수 1
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0

...

총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0

총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0

총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0

총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
총 개수 0
total_reuslt_number =  916


,ID,stores,X,Y,road_address,place_url
0,316685560,인연게스트하우스,126.16475105668802,33.29050181875414,제주특별자치도 제주시 한경면 한장중길 4,http://place.map.kakao.com/316685560
1,20886312,유쾌한게스트하우스,126.163767758902,33.3253652539874,제주특별자치도 제주시 한경면 용수3길 67,http://place.map.kakao.com/20886312
2,376891765,공유게스트하우스,126.17246851736,33.2765353950751,제주특별자치도 서귀포시 대정읍 도원로 20,http://place.map.kakao.com/376891765
3,411980634,오늘밤게스트하우스,126.165632348821,33.2901723443759,제주특별자치도 제주시 한경면 한장중길 14,http://place.map.kakao.com/411980634
4,55932349,별아래게스트하우스,126.165632348821,33.2901723443759,제주특별자치도 제주시 한경면 한장중길 14,http://place.map.kakao.com/55932349


In [10]:
#make_map(df)

In [11]:
df

,ID,stores,X,Y,road_address,place_url
0,316685560,인연게스트하우스,126.16475105668802,33.29050181875414,제주특별자치도 제주시 한경면 한장중길 4,http://place.map.kakao.com/316685560
1,20886312,유쾌한게스트하우스,126.163767758902,33.3253652539874,제주특별자치도 제주시 한경면 용수3길 67,http://place.map.kakao.com/20886312
2,376891765,공유게스트하우스,126.17246851736,33.2765353950751,제주특별자치도 서귀포시 대정읍 도원로 20,http://place.map.kakao.com/376891765
3,411980634,오늘밤게스트하우스,126.165632348821,33.2901723443759,제주특별자치도 제주시 한경면 한장중길 14,http://place.map.kakao.com/411980634
4,55932349,별아래게스트하우스,126.165632348821,33.2901723443759,제주특별자치도 제주시 한경면 한장중길 14,http://place.map.kakao.com/55932349
5,1380861653,제주역게스트하우스,126.165909509602,33.3084261013514,제주특별자치도 제주시 한경면 노을해안로 1171,http://place.map.kakao.com/1380861653
6,344135962,뚜벅이게스트하우스,126.16590494014598,33.30845492168696,제주특별자치도 제주시 한경면 노을해안로 1171,http://place.map.kakao.com/344135962
7,1820720717,스테이스며들다,126.16893327929964,33.32290431333113,제주특별자치도 제주시 한경면 용수2길 24-4,http://place.map.kakao.com/1820720717
8,1690548978,이안게스트하우스,126.1706231165398,33.32907158695217,제주특별자치도 제주시 한경면 용수길 119-5,http://place.map.kakao.com/1690548978
9,27193699,한그루게스트하우스,126.171494931081,33.3372819738248,제주특별자치도 제주시 한경면 주전길 92-54,http://place.map.kakao.com/27193699


In [12]:
df.to_csv('게스트하우스 데이터.csv', index=False, encoding='cp949')